# Data cleaning and processing

In [26]:
import pandas as pd

#defining some helper functions
def col_unique_val(df,colname):
    uniq_vals = []
    for index,row in df.iterrows():
        for col,value in row.items():
            if col == colname and value not in uniq_vals:
                uniq_vals.append(value)
    return uniq_vals

def row_values(df,row_index):
    values = df.loc[row_index].values
    return values

def csv_gen(df,filename):
    headerOBJ = df.columns
    headerD1_2 = []
    for item in headerOBJ:
        headerD1_2.append(item)
    
    # specify the file path and name to extract the different dfs
    filename = filename

    # define the desired formatting options
    format_options = {'sep': ';',  # use semicolon as delimiter
                  'index': False,  # don't include index column
                  'float_format': '%.2f',  # format float values to 2 decimal places
                  'header': headerD1_2,  # rename headers
                  'encoding': 'utf-8'}  # specify encoding type

    # export the dataframe to CSV using the formatting options
    df.to_csv(filename,**format_options) 

## D1 cleaning

Contents:
##### Unit
- **Counts**: raw count of the 
##### Freq
- **A**: Annual
##### Categories of data
- **Total deaths**:
- **Deaths due to external causes: by accident or other causes**
- **Deaths due to external causes: by intentional homicide**
- **Deaths due to external causes: by suicide**
- **Deaths due to natural causes**

In [27]:
D1 = pd.read_csv("../raw_data/D1_deaths_in_prison.csv",header=0)

We observe that there is a stark difference between the `total` death count and the combined number of death with `reason` and one possible explanation is that the total takes into account the _legalally sanctioned deaths_ as well?

In [28]:
D1.head(4)

,UNODC,#ERROR!,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,07/06/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
2,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2005,Counts,231,External
3,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2007,Counts,277,External


In [29]:
#Second line is actaully the column names we need to use
col_names = row_values(D1,1)
print(col_names)

['Iso3_code' 'Country' 'Region' 'Subregion' 'Indicator' 'Dimension'
 'Category' 'Sex' 'Age' 'Year' 'Unit of measurement' 'VALUE' 'Source']


In [30]:
#removing the first row 
D1 = D1.iloc[2:]
D1.head(5)

,UNODC,#ERROR!,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
2,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2005,Counts,231,External
3,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2007,Counts,277,External
4,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2011,Counts,246,External
5,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2013,Counts,240,External
6,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2014,Counts,170,External


In [31]:
#renaming the columns is required
D1.columns = col_names
D1.head(5)

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
2,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2005,Counts,231,External
3,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2007,Counts,277,External
4,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2011,Counts,246,External
5,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2013,Counts,240,External
6,ABW,Aruba,Americas,Latin America and the Caribbean,Persons held,Total,Total,Total,Total,2014,Counts,170,External


In [32]:
regions = []
#Finding all unique regions
regions = col_unique_val(D1,'Region')
print(regions)

#grouping by Region to separate the Data of eurpean countries from the bigger set
D1_group = D1.groupby(['Region'])

#getting the data for only Europe
D1_euro= D1_group.get_group('Europe')
D1_euro

['Americas', 'Asia', 'Africa', 'Europe', 'Oceania']


,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
67,ALB,Albania,Europe,Southern Europe,Persons held,Total,Total,Total,Total,2003,Counts,2561,CTS/External
68,ALB,Albania,Europe,Southern Europe,Persons held,by status,Sentenced,Total,Total,2003,Counts,2071,CTS
69,ALB,Albania,Europe,Southern Europe,Persons held,by status,Unsentenced,Total,Total,2003,Counts,490,CTS/External
70,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,Total,Total,2003,Counts,490,CTS/External
71,ALB,Albania,Europe,Southern Europe,Persons held,Total,Total,Total,Adult,2003,Counts,2524,CTS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57175,SWE,Sweden,Europe,Northern Europe,Persons held,by status,Unsentenced,Male,Total,2020,"Rate per 100,000 population",37.43862112,CTS
57234,UKR,Ukraine,Europe,Eastern Europe,Persons held,Total,Total,Male,Total,2010,"Rate per 100,000 population",510.1761779,Computed
57235,UKR,Ukraine,Europe,Eastern Europe,Persons held,Total,Total,Male,Total,2011,"Rate per 100,000 population",517.8107384,Computed
57236,UKR,Ukraine,Europe,Eastern Europe,Persons held,by status,Unsentenced,Male,Total,2017,"Rate per 100,000 population",88.94136041,CTS


In [33]:
#exploring the values in different columns
#----------------finding all unique values in indicator column
indicator_val = []
indicator_val = col_unique_val(D1_euro,'Indicator')
for item in indicator_val:
    print(item)

Persons held
Persons held unsentenced
Prisons facilities and capacity
Persons entering prison
Mortality in prison
Persons held under other types of supervision


In [34]:
#----------------finding all unique values in category column
catgry_val = []
catgry_val = col_unique_val(D1_euro,'Category')
for item in catgry_val:
    print(item)
    
# after viewing the categories, we see that there are some values like, `Unsentenced for more than 12 months, less than 12 months, 
# less than 6 months`. We need to extract these - to do that, we need to use the Indicator `Person held unsentenced`

Total
Sentenced
Unsentenced
Unsentenced: Total
Official prison capacity
National citizens
Foreign citizens
Intentional Homicide
Acts involving controlled psycho-active substances
Drug possession
Drug trafficking
Deaths due to external causes
Deaths due to external causes: by intentional homicide
Deaths due to external causes: by suicide
Deaths due to external causes: by accident or other causes
Deaths due to natural causes
Rape
Unsentenced for more than 12 months
Number of facilities
Bribery
Unsentenced for less than 12 months
Unsentenced for less than 6 months
Acts leading to death or intending to cause death
Acts leading to harm or intending to cause harm to the person
Injurious acts of a sexual nature
Acts against property involving violence
Acts against property only
Acts involving fraud, deception or corruption
Acts against public order
Acts against public safety and state security
Acts against the natural environment
Other criminal acts not elsewhere classified
Drug Trafficking
A

In [35]:
#grouping the dataframe by Indicator to get data for only `Mortality in Prison`
D1_temp = D1_euro.groupby(['Indicator'])
D1_mortality = D1_temp.get_group('Mortality in prison')
D1_mortality.head(5)

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
186,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2013,Counts,13,CTS
187,ALB,Albania,Europe,Southern Europe,Mortality in prison,by type of death,Deaths due to external causes,Total,Total,2013,Counts,9,CTS
188,ALB,Albania,Europe,Southern Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,2013,Counts,0,CTS
189,ALB,Albania,Europe,Southern Europe,Mortality in prison,by type of death,Deaths due to external causes: by suicide,Total,Total,2013,Counts,4,CTS
206,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2014,Counts,12,CTS


In [36]:
dim = []
dim = col_unique_val(D1_mortality,'Dimension')
print(dim)
D1_mortality.columns

['Total', 'by type of death']


Index(['Iso3_code', 'Country', 'Region', 'Subregion', 'Indicator', 'Dimension',
       'Category', 'Sex', 'Age', 'Year', 'Unit of measurement', 'VALUE',
       'Source'],
      dtype='object')

In [37]:
D1_groupedCategory = D1_mortality.groupby(['Category'])
print(type(D1_groupedCategory.groups))
for value in D1_groupedCategory.groups:
    print('---',value)

<class 'pandas.io.formats.printing.PrettyDict'>
--- Deaths due to external causes
--- Deaths due to external causes: by accident or other causes
--- Deaths due to external causes: by intentional homicide
--- Deaths due to external causes: by suicide
--- Deaths due to natural causes
--- Total


In [38]:
#creating D1_totalcount
D1_total = D1_groupedCategory.get_group('Total')
D1_total.head(10)

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
186,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2013,Counts,13,CTS
206,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2014,Counts,12,CTS
224,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2015,Counts,11,CTS
244,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2016,Counts,17,CTS
267,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2017,Counts,23,CTS
296,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2018,Counts,24,CTS
326,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2019,Counts,27,CTS
347,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2020,Counts,26,CTS
403,AND,Andorra,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2010,Counts,0,CTS
420,AND,Andorra,Europe,Southern Europe,Mortality in prison,Total,Total,Total,Total,2011,Counts,0,CTS


> Note:
    In the D1_total dataframe we have two columns namely, `Dimension` & `Category` which both have the same value and only one value but I have choosen to not get rid of it to aid the future user incase of using the dataframe with other dataframes. These columns provide a context.

In [39]:
#creating D1_reason
D1_reason1 = D1_groupedCategory.get_group('Deaths due to external causes: by accident or other causes')
D1_reason2 = D1_groupedCategory.get_group('Deaths due to external causes: by intentional homicide')
D1_reason3 = D1_groupedCategory.get_group('Deaths due to external causes: by suicide')
D1_reason4 = D1_groupedCategory.get_group('Deaths due to natural causes')

#D1_reason_temp1 = pd.concat([D1_reason1,D1_reason2])
#D1_reason_temp2 = pd.concat([D1_reason3,D1_reason4])
#D1_reason = pd.concat([D1_reason_temp1,D1_reason_temp2])

#droping `Sex` and `Age` which not from the mortality data and also droping `Dimension` as it is self-evident
D1_total = D1_total.drop(columns=['Sex','Age','Dimension'])
D1_total.head()

,Iso3_code,Country,Region,Subregion,Indicator,Category,Year,Unit of measurement,VALUE,Source
186,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,2013,Counts,13,CTS
206,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,2014,Counts,12,CTS
224,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,2015,Counts,11,CTS
244,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,2016,Counts,17,CTS
267,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,2017,Counts,23,CTS


In [40]:
#performing the transform now
df1_strncols = D1_total.drop(columns=['Year','VALUE'])
df1_strncols = df1_strncols.drop_duplicates()
df1_transformed1 = D1_total.pivot(index = 'Country',columns='Year', values='VALUE')
df1_transformed1 = df1_transformed1.reset_index()
#now merging the yearly values with the strng data

D1_total = pd.merge(df1_strncols,df1_transformed1,on='Country')
D1_total = D1_total.fillna(":")
D1_total

,Iso3_code,Country,Region,Subregion,Indicator,Category,Unit of measurement,Source,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ALB,Albania,Europe,Southern Europe,Mortality in prison,Total,Counts,CTS,:,:,:,13,12,11,17,23,24,27,26,:
1,AND,Andorra,Europe,Southern Europe,Mortality in prison,Total,Counts,CTS,0,0,1,0,0,0,1,1,:,:,:,:
2,AUT,Austria,Europe,Western Europe,Mortality in prison,Total,Counts,CTS,35,37,33,39,23,34,40,33,44,38,29,:
3,BEL,Belgium,Europe,Western Europe,Mortality in prison,Total,Counts,CTS,:,:,:,:,59,44,:,44,42,39,48,:
4,BGR,Bulgaria,Europe,Eastern Europe,Mortality in prison,Total,Counts,CTS,46,50,52,32,28,43,48,36,32,49,52,:
5,BIH,Bosnia and Herzegovina,Europe,Southern Europe,Mortality in prison,Total,Counts,CTS/SDG,:,:,:,:,:,8,8,:,:,:,23,:
6,BLR,Belarus,Europe,Eastern Europe,Mortality in prison,Total,Counts,CTS,:,:,109,87,96,:,:,:,:,:,:,:
7,CHE,Switzerland,Europe,Western Europe,Mortality in prison,Total,Counts,CTS,12,10,29,10,15,24,17,18,12,18,9,:
8,CZE,Czechia,Europe,Eastern Europe,Mortality in prison,Total,Counts,CTS,35,31,38,26,31,43,43,37,41,43,40,:
9,DEU,Germany,Europe,Western Europe,Mortality in prison,Total,Counts,CTS,:,:,:,:,152,146,163,173,:,:,:,:


In [41]:
#this is commented out to stop it from generating a csv 
'''
headerOBJ = D1_total.columns
headerD1_2 = []
for item in headerOBJ:
    headerD1_2.append(item)
    
# specify the file path and name to extract the different dfs
filename = 'D1.1.csv'

# define the desired formatting options
format_options = {'sep': ';',  # use semicolon as delimiter
                  'index': False,  # don't include index column
                  'float_format': '%.2f',  # format float values to 2 decimal places
                  'header': headerD1_2,  # rename headers
                  'encoding': 'utf-8'}  # specify encoding type

# export the dataframe to CSV using the formatting options
D1_total.to_csv(filename,**format_options) 
'''

"\nheaderOBJ = D1_total.columns\nheaderD1_2 = []\nfor item in headerOBJ:\n    headerD1_2.append(item)\n    \n# specify the file path and name to extract the different dfs\nfilename = 'D1.1.csv'\n\n# define the desired formatting options\nformat_options = {'sep': ';',  # use semicolon as delimiter\n                  'index': False,  # don't include index column\n                  'float_format': '%.2f',  # format float values to 2 decimal places\n                  'header': headerD1_2,  # rename headers\n                  'encoding': 'utf-8'}  # specify encoding type\n\n# export the dataframe to CSV using the formatting options\nD1_total.to_csv(filename,**format_options) \n"

In [42]:
D1_reason = D1_reason.drop(columns=['Sex','Age','Dimension'])
print(col_unique_val(D1_reason,'Category'))

['Deaths due to external causes: by accident or other causes', 'Deaths due to external causes: by intentional homicide', 'Deaths due to external causes: by suicide', 'Deaths due to natural causes']


In [43]:
#d1reasongrp = D1_reason.groupby('Category')
#D1_reason1 = d1reasongrp.get_group('Deaths due to external causes: by accident or other causes')
#D1_reason2 = d1reasongrp.get_group('Deaths due to external causes: by intentional homicide')
#D1_reason3 = d1reasongrp.get_group('Deaths due to external causes: by suicide')
#D1_reason4 = d1reasongrp.get_group('Deaths due to natural causes')
D1_reason4.tail()

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
23949,SWE,Sweden,Europe,Northern Europe,Mortality in prison,by type of death,Deaths due to natural causes,Total,Total,2016,Counts,7,CTS
24025,SWE,Sweden,Europe,Northern Europe,Mortality in prison,by type of death,Deaths due to natural causes,Total,Total,2017,Counts,7,CTS
24081,SWE,Sweden,Europe,Northern Europe,Mortality in prison,by type of death,Deaths due to natural causes,Total,Total,2018,Counts,12,CTS
24138,SWE,Sweden,Europe,Northern Europe,Mortality in prison,by type of death,Deaths due to natural causes,Total,Total,2019,Counts,6,CTS
24165,SWE,Sweden,Europe,Northern Europe,Mortality in prison,by type of death,Deaths due to natural causes,Total,Total,2020,Counts,4,CTS


In [22]:
#performing transform
df1_strnR1 = D1_reason1.drop(columns=['Year','VALUE'])
df1_strnR1 = df1_strnR1.drop_duplicates()
df1_transR1 = D1_reason1.pivot(index = 'Country',columns='Year', values='VALUE')
df1_transR1 = df1_transR1.reset_index()
#performing merge
D1_reason1 = pd.merge(df1_strnR1,df1_transR1,on='Country')
D1_reason1 = D1_reason1.fillna(":")

#performing transform
df1_strnR2 = D1_reason2.drop(columns=['Year','VALUE'])
df1_strnR2 = df1_strnR2.drop_duplicates()
df1_transR2 = D1_reason2.pivot(index = 'Country',columns='Year', values='VALUE')
df1_transR2 = df1_transR2.reset_index()
#performing merge
D1_reason2 = pd.merge(df1_strnR2,df1_transR2,on='Country')
D1_reason2 = D1_reason2.fillna(":")

#performing transform
df1_strnR3 = D1_reason3.drop(columns=['Year','VALUE'])
df1_strnR3 = df1_strnR3.drop_duplicates()
df1_transR3 = D1_reason3.pivot(index = 'Country',columns='Year', values='VALUE')
df1_transR3 = df1_transR3.reset_index()
#performing merge
D1_reason3 = pd.merge(df1_strnR3,df1_transR3,on='Country')
D1_reason3 = D1_reason3.fillna(":")

#performing transform
df1_strnR4 = D1_reason4.drop(columns=['Year','VALUE'])
df1_strnR4 = df1_strnR4.drop_duplicates()
df1_transR4 = D1_reason4.pivot(index = 'Country',columns='Year', values='VALUE')
df1_transR4 = df1_transR4.reset_index()
#performing merge
D1_reason4 = pd.merge(df1_strnR4,df1_transR4,on='Country')
D1_reason4 = D1_reason4.fillna(":")

csv_gen(D1_reason1,'D1.21.csv')
csv_gen(D1_reason2,'D1.22.csv')
csv_gen(D1_reason3,'D1.23.csv')
csv_gen(D1_reason4,'D1.24.csv')

D1_reason2

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Unit of measurement,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ALB,Albania,Europe,Southern Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,Counts,...,:,0,0,0,0,0,0,0,0,:
1,AND,Andorra,Europe,Southern Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,Counts,...,0,0,0,0,:,:,:,:,:,:
2,AUT,Austria,Europe,Western Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,Counts,...,:,:,:,0,0,1,0,0,1,:
3,BGR,Bulgaria,Europe,Eastern Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,Counts,...,:,:,:,0,0,0,0,0,0,:
4,CZE,Czechia,Europe,Eastern Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,Counts,...,:,0,1,0,0,0,0,0,0,:
5,DNK,Denmark,Europe,Northern Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,Counts,...,0,0,0,0,0,:,:,:,:,:
6,ESP,Spain,Europe,Southern Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,Counts,...,:,:,3,1,0,1,1,0,2,3
7,EST,Estonia,Europe,Northern Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,Counts,...,:,0,0,0,:,:,:,:,:,:
8,FIN,Finland,Europe,Northern Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,Counts,...,:,0,1,0,0,0,0,0,0,:
9,FRA,France,Europe,Western Europe,Mortality in prison,by type of death,Deaths due to external causes: by intentional ...,Total,Total,Counts,...,:,1,1,3,6,3,1,:,:,:


In [44]:
#commneted out to stop csv generation
'''
headerOBJ = D1_reason.columns
headerD1_2 = []
for item in headerOBJ:
    headerD1_2.append(item)
    
# specify the file path and name to extract the different dfs
filename = 'D1.12.csv'

# define the desired formatting options
format_options = {'sep': ';',  # use semicolon as delimiter
                  'index': False,  # don't include index column
                  'float_format': '%.2f',  # format float values to 2 decimal places
                  'header': headerD1_2,  # rename headers
                  'encoding': 'utf-8'}  # specify encoding type

# export the dataframe to CSV using the formatting options
D1_reason.to_csv(filename,**format_options) 
'''

"\nheaderOBJ = D1_reason.columns\nheaderD1_2 = []\nfor item in headerOBJ:\n    headerD1_2.append(item)\n    \n# specify the file path and name to extract the different dfs\nfilename = 'D1.12.csv'\n\n# define the desired formatting options\nformat_options = {'sep': ';',  # use semicolon as delimiter\n                  'index': False,  # don't include index column\n                  'float_format': '%.2f',  # format float values to 2 decimal places\n                  'header': headerD1_2,  # rename headers\n                  'encoding': 'utf-8'}  # specify encoding type\n\n# export the dataframe to CSV using the formatting options\nD1_reason.to_csv(filename,**format_options) \n"

### <span style='color:red'>Important !</span>
Note that for now, the process of grouping the data with resepct to the `category` and `indicator` has given us a dataframe which have the same number of rows for each country which shows that we have **different time spans** for different countries and to use them together, it would be a good idea to first create new dataframes for each country with columns having a placeholder for the years for which ther is not data. This way we also _normalise_ this dataset with the others being used as they use a placeholder for missing data.

<hr>

## D2 cleaning

Contents:
##### Unit
- **Counts**: raw count of the people in prison
##### Freq
- **A**: Annual
##### Categories of data
- **Total persons held**:
- **Male persons held**
- **Female persons held**
- **Unsentenced for less than 12 months**
- **Unsentenced for less than 6 months**
- **Unsentenced for more than 12 months**
- **Unsentenced: Total**

In [45]:
for item in indicator_val:
    print(item)

Persons held
Persons held unsentenced
Prisons facilities and capacity
Persons entering prison
Mortality in prison
Persons held under other types of supervision


## CREATING FIRST PART OF D2 – TOTAL HELD

In [49]:
#grouping the dataframe by Indicator to get data for only `Persons held`
D2_persHeld = D1_temp.get_group('Persons held')
D2_persHeld.head(20)

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
67,ALB,Albania,Europe,Southern Europe,Persons held,Total,Total,Total,Total,2003,Counts,2561,CTS/External
68,ALB,Albania,Europe,Southern Europe,Persons held,by status,Sentenced,Total,Total,2003,Counts,2071,CTS
69,ALB,Albania,Europe,Southern Europe,Persons held,by status,Unsentenced,Total,Total,2003,Counts,490,CTS/External
71,ALB,Albania,Europe,Southern Europe,Persons held,Total,Total,Total,Adult,2003,Counts,2524,CTS
72,ALB,Albania,Europe,Southern Europe,Persons held,Total,Total,Total,Juveniles,2003,Counts,37,CTS
76,ALB,Albania,Europe,Southern Europe,Persons held,by status,Sentenced,Total,Total,2004,Counts,2321,CTS
77,ALB,Albania,Europe,Southern Europe,Persons held,by status,Unsentenced,Total,Total,2004,Counts,623,CTS/External
79,ALB,Albania,Europe,Southern Europe,Persons held,Total,Total,Total,Adult,2004,Counts,2299,CTS
80,ALB,Albania,Europe,Southern Europe,Persons held,Total,Total,Total,Juveniles,2004,Counts,22,CTS
84,ALB,Albania,Europe,Southern Europe,Persons held,Total,Total,Total,Total,2004,Counts,2944,CTS/External


In [50]:
for item in (col_unique_val(D2_persHeld,'Dimension')):
    print(item)
    
#we only want total so lets group it by dimension = total

D2_persHeldTotal = D2_persHeld.groupby(['Dimension'])
D2_persHeld = D2_persHeldTotal.get_group('Total')

# droping the dimension column now
D2_persHeld = D2_persHeld.drop(columns=['Dimension','Category'])

#since we dont care to differentiate between by Age, we are only getting the data from the Total rows of the age which contains both the Adults and juveniles.
D2_persHeldAgeTotal = D2_persHeld.groupby(['Age'])
D2_persHeld = D2_persHeldAgeTotal.get_group('Total')
D2_persHeld = D2_persHeld.drop(columns=['Age'])

# now we will remove the data in the rate/100000 population and only keep raw count
D2_persHeldUnitgroup = D2_persHeld.groupby(['Unit of measurement'])
D2_persHeld = D2_persHeldUnitgroup.get_group('Counts')
D2_persHeld = D2_persHeld.reset_index(drop=True)
D2_persHeld

Total
by status
by citizenship
by selected crime
by type of criminal acts


,Iso3_code,Country,Region,Subregion,Indicator,Sex,Year,Unit of measurement,VALUE,Source
0,ALB,Albania,Europe,Southern Europe,Persons held,Total,2003,Counts,2561,CTS/External
1,ALB,Albania,Europe,Southern Europe,Persons held,Total,2004,Counts,2944,CTS/External
2,ALB,Albania,Europe,Southern Europe,Persons held,Total,2005,Counts,3464,CTS/External
3,ALB,Albania,Europe,Southern Europe,Persons held,Total,2006,Counts,3798,CTS/External
4,ALB,Albania,Europe,Southern Europe,Persons held,Total,2007,Counts,4555,CTS/External
...,...,...,...,...,...,...,...,...,...,...
2026,VAT,Holy See,Europe,Southern Europe,Persons held,Male,2011,Counts,0,Computed
2027,VAT,Holy See,Europe,Southern Europe,Persons held,Male,2012,Counts,1,Computed
2028,VAT,Holy See,Europe,Southern Europe,Persons held,Male,2013,Counts,0,Computed
2029,VAT,Holy See,Europe,Southern Europe,Persons held,Male,2014,Counts,0,Computed


In [51]:
#store the repeated string values in a different df
D2_persHeldStrng = D2_persHeld.drop(columns=['Year','VALUE'])
D2_persHeldStrng = D2_persHeldStrng.reset_index(drop=True)
D2_persHeldStrng

,Iso3_code,Country,Region,Subregion,Indicator,Sex,Unit of measurement,Source
0,ALB,Albania,Europe,Southern Europe,Persons held,Total,Counts,CTS/External
1,ALB,Albania,Europe,Southern Europe,Persons held,Total,Counts,CTS/External
2,ALB,Albania,Europe,Southern Europe,Persons held,Total,Counts,CTS/External
3,ALB,Albania,Europe,Southern Europe,Persons held,Total,Counts,CTS/External
4,ALB,Albania,Europe,Southern Europe,Persons held,Total,Counts,CTS/External
...,...,...,...,...,...,...,...,...
2026,VAT,Holy See,Europe,Southern Europe,Persons held,Male,Counts,Computed
2027,VAT,Holy See,Europe,Southern Europe,Persons held,Male,Counts,Computed
2028,VAT,Holy See,Europe,Southern Europe,Persons held,Male,Counts,Computed
2029,VAT,Holy See,Europe,Southern Europe,Persons held,Male,Counts,Computed


In [52]:
#performing the necessary separation by sex for the strings here
D2_persHeldStrnggrp = D2_persHeldStrng.groupby(['Sex'])

D2_persHeldStrng1 = D2_persHeldStrnggrp.get_group('Total')
D2_persHeldStrng1 = D2_persHeldStrng1.drop_duplicates()
D2_persHeldStrng2 = D2_persHeldStrnggrp.get_group('Male')
D2_persHeldStrng2 = D2_persHeldStrng2.drop_duplicates()
D2_persHeldStrng3 = D2_persHeldStrnggrp.get_group('Female')
D2_persHeldStrng3 = D2_persHeldStrng3.drop_duplicates()
D2_persHeldStrng3

,Iso3_code,Country,Region,Subregion,Indicator,Sex,Unit of measurement,Source
835,ALB,Albania,Europe,Southern Europe,Persons held,Female,Counts,Computed
851,AND,Andorra,Europe,Southern Europe,Persons held,Female,Counts,Computed
864,AUT,Austria,Europe,Western Europe,Persons held,Female,Counts,Computed/WPB-ICPR
879,BEL,Belgium,Europe,Western Europe,Persons held,Female,Counts,WPB-ICPR/Computed
881,BGR,Bulgaria,Europe,Eastern Europe,Persons held,Female,Counts,Computed
898,BIH,Bosnia and Herzegovina,Europe,Southern Europe,Persons held,Female,Counts,Computed
901,BLR,Belarus,Europe,Eastern Europe,Persons held,Female,Counts,Computed/WPB-ICPR
915,CHE,Switzerland,Europe,Western Europe,Persons held,Female,Counts,Computed
930,CZE,Czechia,Europe,Eastern Europe,Persons held,Female,Counts,Computed
946,DEU,Germany,Europe,Western Europe,Persons held,Female,Counts,WPB-ICPR


In [53]:
#IMPORTANT - this transform gives the values of the subsequent years in the subsequent rows. Maybe to solve this, we need to use index for which we need to
# first separate the values for different sexes
D2_temppsex = D2_persHeld.groupby(['Sex'])

D2_temtotal = D2_temppsex.get_group('Total')
D2_temMale = D2_temppsex.get_group('Male')
D2_temFemale = D2_temppsex.get_group('Female')

# perform a transformation
df_transformed1 = D2_temtotal.pivot(index = 'Country',columns='Year', values='VALUE')
df_transformed2 = D2_temMale.pivot(index = 'Country',columns='Year', values='VALUE')
df_transformed3 = D2_temFemale.pivot(index = 'Country',columns='Year', values='VALUE')
df_transformed2.head()

Year,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Country,,,,,,,,,,,,,,,,,,,
Albania,NaN,2262,2526,2778,2706,2965,4551,4559,4575,4536,4909,5580,5860,5933,5581,5207,4959,NaN,NaN
Andorra,NaN,NaN,NaN,NaN,63,60,45,36,42,29,37,41,32,42,38,45,43,NaN,NaN
Austria,NaN,NaN,7771,7886,7648,7011,7625,7752,7882,8226,8318,8185,NaN,8127,8334,8600,8466,NaN,NaN
Belarus,35789,40488,39263,42823,43714,41399,37277,36639,33982,26901,26485,27544,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10355,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
D2_persHeldStrng1 = D2_persHeldStrng1.reset_index(drop=True)
D2_persHeldStrng2 = D2_persHeldStrng2.reset_index(drop=True)
D2_persHeldStrng3 = D2_persHeldStrng3.reset_index(drop=True)

df_transformed1 = df_transformed1.reset_index(drop=True)
df_transformed2 = df_transformed2.reset_index(drop=True)
df_transformed3 = df_transformed3.reset_index(drop=True)

In [73]:
#perform merge of the strng df with the value df
D2_totalPersHeldT = pd.merge(D2_persHeldStrng1, df_transformed1, left_index=True, right_index=True)
D2_totalPersHeldM = pd.merge(D2_persHeldStrng2, df_transformed2, left_index=True, right_index=True)
D2_totalPersHeldF = pd.merge(D2_persHeldStrng3, df_transformed3, left_index=True, right_index=True)

D2_totalPersHeldT
#still need to remove the NAN values and repeat the operation for all the sexes.

,Iso3_code,Country,Region,Subregion,Indicator,Sex,Unit of measurement,Source,2003,2004,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ALB,Albania,Europe,Southern Europe,Persons held,Total,Counts,CTS/External,2561,2944,...,4618,4998,5689,5981,6031,5674,5316,5045,4614,NaN
1,AND,Andorra,Europe,Southern Europe,Persons held,Total,Counts,External/WPB-ICPR/CTS,61,61,...,36,43,53,41,46,43,49,49,50,NaN
2,AUT,Austria,Europe,Western Europe,Persons held,Total,Counts,CTS,NaN,NaN,...,8805,8862,8692,NaN,8619,8852,9163,9072,8488,NaN
3,BEL,Belgium,Europe,Western Europe,Persons held,Total,Counts,External/CTS,48434,42253,...,28841,28471,29776,33329,35169,34356,32556,NaN,NaN,NaN
4,BGR,Bulgaria,Europe,Eastern Europe,Persons held,Total,Counts,CTS,NaN,9245,...,11212,12126,11769,10994,11071,10471,10261,10559,10381,NaN
5,BIH,Bosnia and Herzegovina,Europe,Southern Europe,Persons held,Total,Counts,External/CTS,NaN,2405,...,2928,2898,2825,2730,2832,2820,4377,4482,4332,NaN
6,BLR,Belarus,Europe,Eastern Europe,Persons held,Total,Counts,External/CTS,10066,10871,...,9493,8834,7870,7408,7345,6988,6651,6448,6251,NaN
7,CHE,Switzerland,Europe,Western Europe,Persons held,Total,Counts,External/CTS,2805,3029,...,4741,4352,3763,3341,3108,3190,3217,3533,3531,NaN
8,CZE,Czechia,Europe,Eastern Europe,Persons held,Total,Counts,External/CTS,17277,18343,...,22644,16645,18658,20866,22481,22159,21577,21048,19286,NaN
9,DEU,Germany,Europe,Western Europe,Persons held,Total,Counts,CTS,3641,3767,...,3829,4091,3583,3203,3408,3418,3635,3920,4166,NaN


In [75]:

D2_totalPersHeldT = D2_totalPersHeldT.fillna(":")
D2_totalPersHeldM = D2_totalPersHeldM.fillna(":")
D2_totalPersHeldF = D2_totalPersHeldF.fillna(":")
D2_totalPersHeldT

#REMEBER that there are 3 dataframes for D1 - which can be accessed by changing the Last letter of the variable D2_totalPersHeld, for example
# D2_totalPersHeldM is for the data of male prisoners held, and similarly F for female and T for total.

,Iso3_code,Country,Region,Subregion,Indicator,Sex,Unit of measurement,Source,2003,2004,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ALB,Albania,Europe,Southern Europe,Persons held,Total,Counts,CTS/External,2561,2944,...,4618,4998,5689,5981,6031,5674,5316,5045,4614,:
1,AND,Andorra,Europe,Southern Europe,Persons held,Total,Counts,External/WPB-ICPR/CTS,61,61,...,36,43,53,41,46,43,49,49,50,:
2,AUT,Austria,Europe,Western Europe,Persons held,Total,Counts,CTS,:,:,...,8805,8862,8692,:,8619,8852,9163,9072,8488,:
3,BEL,Belgium,Europe,Western Europe,Persons held,Total,Counts,External/CTS,48434,42253,...,28841,28471,29776,33329,35169,34356,32556,:,:,:
4,BGR,Bulgaria,Europe,Eastern Europe,Persons held,Total,Counts,CTS,:,9245,...,11212,12126,11769,10994,11071,10471,10261,10559,10381,:
5,BIH,Bosnia and Herzegovina,Europe,Southern Europe,Persons held,Total,Counts,External/CTS,:,2405,...,2928,2898,2825,2730,2832,2820,4377,4482,4332,:
6,BLR,Belarus,Europe,Eastern Europe,Persons held,Total,Counts,External/CTS,10066,10871,...,9493,8834,7870,7408,7345,6988,6651,6448,6251,:
7,CHE,Switzerland,Europe,Western Europe,Persons held,Total,Counts,External/CTS,2805,3029,...,4741,4352,3763,3341,3108,3190,3217,3533,3531,:
8,CZE,Czechia,Europe,Eastern Europe,Persons held,Total,Counts,External/CTS,17277,18343,...,22644,16645,18658,20866,22481,22159,21577,21048,19286,:
9,DEU,Germany,Europe,Western Europe,Persons held,Total,Counts,CTS,3641,3767,...,3829,4091,3583,3203,3408,3418,3635,3920,4166,:


In [ ]:
#commented to stop export of data to csv
'''
headerOBJ = D2_totalPersHeldM.columns
headerD2_2 = []
for item in headerOBJ:
    headerD2_2.append(item)
    
# specify the file path and name to extract the different dfs
filename = 'D2.12.csv'

# define the desired formatting options
format_options = {'sep': ';',  # use semicolon as delimiter
                  'index': False,  # don't include index column
                  'float_format': '%.2f',  # format float values to 2 decimal places
                  'header': headerD2_2,  # rename headers
                  'encoding': 'utf-8'}  # specify encoding type

# export the dataframe to CSV using the formatting options
D2_totalPersHeldF.to_csv(filename,**format_options) 
'''

## CREATING SECOND PART OF D2 – HELD UNSENTENCED WITH LENGTH OF DETENTION

In [77]:
#==============starting D2.2=====================
D2_persHeldUn = D1_temp.get_group('Persons held unsentenced')
D2_persHeldUn.describe()

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
count,3225,3225,3225,3225,3225,3225,3225,3225,3225,3225,3225,3225,2029
unique,51,51,1,4,1,1,5,3,1,19,3,2970,12
top,FIN,Finland,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,Total,Total,2017,Percentage of total persons held,0,CTS/External
freq,118,118,3225,1068,3225,3225,1496,2796,3225,301,1196,31,744


We observer 3 different Units of measurement, lets see what they are and select only one - raw count.

In [78]:
print(col_unique_val(D2_persHeldUn,'Unit of measurement'))

['Counts', 'Percentage of total persons held', 'Rate per 100,000 population']


In [79]:
D2_persHeldUnCountgrp = D2_persHeldUn.groupby(['Unit of measurement'])
D2_persHeldCount = D2_persHeldUnCountgrp.get_group('Counts')
D2_persHeldCount.describe()

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
count,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
unique,51,51,1,4,1,1,4,1,1,19,1,875,12
top,FIN,Finland,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,Total,Total,2017,Counts,0,CTS/External
freq,42,42,1069,358,1069,1069,788,1069,1069,99,1069,13,372


Now we have only four categories for the data which are following:
- Unsentenced for less than 12 months
- Unsentenced for less than 6 months
- Unsentenced for more than 12 months
- Unsentenced Total

We also see that the `Sex` and `Age` columns have non-relavant values so we will be dropping them.

In [80]:
D2_persHeldCount = D2_persHeldCount.drop(columns=['Sex','Age'])
D2_persHeldCount

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Year,Unit of measurement,VALUE,Source
70,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,2003,Counts,490,CTS/External
78,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,2004,Counts,623,CTS/External
87,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,2005,Counts,872,CTS/External
98,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,2006,Counts,941,CTS/External
109,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,2007,Counts,1765,CTS/External
...,...,...,...,...,...,...,...,...,...,...,...
25788,VAT,Holy See,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,2011,Counts,0,CTS
25799,VAT,Holy See,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,2012,Counts,1,CTS
25820,VAT,Holy See,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,2013,Counts,0,CTS
25841,VAT,Holy See,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced: Total,2014,Counts,0,CTS


Now we will separate the repatitive string data from the numeric data for the four categories mentioned above and we will also group the the category before we transform the df to avoid the error of duplicate values.

In [81]:
D2_persHeldCountCategrp = D2_persHeldCount.groupby(['Category'])
D2_persHeldCat1 = D2_persHeldCountCategrp.get_group('Unsentenced for less than 12 months')
D2_persHeldCat2 = D2_persHeldCountCategrp.get_group('Unsentenced for less than 6 months')
D2_persHeldCat3 = D2_persHeldCountCategrp.get_group('Unsentenced for more than 12 months')
D2_persHeldCat4 = D2_persHeldCountCategrp.get_group('Unsentenced: Total')

#the dataframe at this point is alright
D2_persHeldCat2

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Year,Unit of measurement,VALUE,Source
295,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,2018,Counts,1443,CTS
310,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,2019,Counts,1465,CTS
346,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,2020,Counts,1447,CTS
503,AND,Andorra,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,2015,Counts,4,CTS
513,AND,Andorra,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,2016,Counts,12,CTS
...,...,...,...,...,...,...,...,...,...,...,...
23393,SVK,Slovakia,Europe,Eastern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,2018,Counts,1116,CTS
23421,SVK,Slovakia,Europe,Eastern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,2019,Counts,1186,CTS
23450,SVK,Slovakia,Europe,Eastern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,2020,Counts,1157,CTS
24118,SWE,Sweden,Europe,Northern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,2019,Counts,9332,CTS


In [82]:
#separate the strng part with the numeric values
D2_persHeldCat1Strng = D2_persHeldCat1.drop(columns=['Year','VALUE']) 
D2_persHeldCat2Strng = D2_persHeldCat2.drop(columns=['Year','VALUE'])
D2_persHeldCat3Strng = D2_persHeldCat3.drop(columns=['Year','VALUE'])
D2_persHeldCat4Strng = D2_persHeldCat4.drop(columns=['Year','VALUE'])


D2_persHeldCat1Strng = D2_persHeldCat1Strng.drop_duplicates()
D2_persHeldCat2Strng = D2_persHeldCat2Strng.drop_duplicates()
D2_persHeldCat3Strng = D2_persHeldCat3Strng.drop_duplicates()
D2_persHeldCat4Strng = D2_persHeldCat4Strng.drop_duplicates()
#D2_persHeldCat1Strng = D2_persHeldCat1Strng.reset_index(drop=True)
D2_persHeldCat1Strng
#peform transformation to get the time series structure

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Unit of measurement,Source
294,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS
502,AND,Andorra,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS
1535,AUT,Austria,Europe,Western Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS
2759,BIH,Bosnia and Herzegovina,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS/SDG
6156,CZE,Czechia,Europe,Eastern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS
7548,ESP,Spain,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS/SDG
8105,FIN,Finland,Europe,Northern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS
8478,FRA,France,Europe,Western Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS
9051,GBR_NI,United Kingdom (Northern Ireland),Europe,Northern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS
11150,HRV,Croatia,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS


In [83]:
#performing tranformation on all grouped categories
D2_persHeldCat1num = D2_persHeldCat1.pivot(index = 'Country',columns='Year', values='VALUE')
D2_persHeldCat1num = D2_persHeldCat1num.reset_index()
D2_persHeldCat2num = D2_persHeldCat2.pivot(index = 'Country',columns='Year', values='VALUE')
D2_persHeldCat2num = D2_persHeldCat2num.reset_index()
D2_persHeldCat3num = D2_persHeldCat3.pivot(index = 'Country',columns='Year', values='VALUE')
D2_persHeldCat3num = D2_persHeldCat3num.reset_index()
D2_persHeldCat4num = D2_persHeldCat4.pivot(index = 'Country',columns='Year', values='VALUE')
D2_persHeldCat4num = D2_persHeldCat4num.reset_index()


D2_persHeldCat2num

Year,Country,2013,2014,2015,2016,2017,2018,2019,2020
0,Albania,NaN,NaN,NaN,NaN,NaN,1443,1465,1447
1,Andorra,NaN,NaN,4,12,12,13,7,14
2,Austria,NaN,NaN,NaN,1453,1559,1537,1480,1276
3,Bosnia and Herzegovina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,853
4,Croatia,NaN,NaN,NaN,NaN,713,758,969,986
5,Czechia,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
6,Finland,396,458,396,428,429,358,442,425
7,France,NaN,NaN,NaN,8725,8901,8533,8814,7864
8,Hungary,NaN,NaN,NaN,NaN,1757,1353,3335,3227
9,Italy,7649,6712,5988,6656,6901,7128,7040,5845


In [84]:
#performing merge on the country column
D2_totalPersHeldCat1 = pd.merge(D2_persHeldCat1Strng, D2_persHeldCat1num, on='Country')
D2_totalPersHeldCat2 = pd.merge(D2_persHeldCat2Strng, D2_persHeldCat2num, on='Country')
D2_totalPersHeldCat3 = pd.merge(D2_persHeldCat3Strng, D2_persHeldCat3num, on='Country')
D2_totalPersHeldCat4 = pd.merge(D2_persHeldCat4Strng, D2_persHeldCat4num, on='Country')

D2_totalPersHeldCat2

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Unit of measurement,Source,2013,2014,2015,2016,2017,2018,2019,2020
0,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,Counts,CTS,NaN,NaN,NaN,NaN,NaN,1443,1465,1447
1,AND,Andorra,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,Counts,CTS,NaN,NaN,4,12,12,13,7,14
2,AUT,Austria,Europe,Western Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,Counts,CTS,NaN,NaN,NaN,1453,1559,1537,1480,1276
3,BIH,Bosnia and Herzegovina,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,Counts,CTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,853
4,CZE,Czechia,Europe,Eastern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,Counts,CTS,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
5,ESP,Spain,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,Counts,CTS,NaN,NaN,NaN,3508,3423,3965,3967,3671
6,FIN,Finland,Europe,Northern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,Counts,CTS,396,458,396,428,429,358,442,425
7,FRA,France,Europe,Western Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,Counts,CTS,NaN,NaN,NaN,8725,8901,8533,8814,7864
8,GBR_NI,United Kingdom (Northern Ireland),Europe,Northern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,Counts,CTS,NaN,306,253,248,288,336,NaN,367
9,HRV,Croatia,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 6 months,Counts,CTS,NaN,NaN,NaN,NaN,713,758,969,986


In [85]:
#replacing the NAN values with the `:` string values
D2_totalPersHeldCat1 = D2_totalPersHeldCat1.fillna(":")
D2_totalPersHeldCat2 = D2_totalPersHeldCat2.fillna(":")
D2_totalPersHeldCat3 = D2_totalPersHeldCat3.fillna(":")
D2_totalPersHeldCat4 = D2_totalPersHeldCat4.fillna(":")
D2_totalPersHeldCat1

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Unit of measurement,Source,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ALB,Albania,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS,:,:,:,:,:,1984,2011,1936,:
1,AND,Andorra,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS,:,:,4,19,18,19,6,6,:
2,AUT,Austria,Europe,Western Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS,:,:,:,1678,1846,1797,1732,1461,:
3,BIH,Bosnia and Herzegovina,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS/SDG,:,:,:,:,:,1094,1137,1074,:
4,CZE,Czechia,Europe,Eastern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS,:,:,:,0,:,:,:,:,:
5,ESP,Spain,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS/SDG,:,:,:,5224,5207,5716,5852,5100,5481
6,FIN,Finland,Europe,Northern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS,506,573,530,552,557,487,588,573,:
7,FRA,France,Europe,Western Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS,:,:,:,13066,13076,13489,13923,11480,:
8,GBR_NI,United Kingdom (Northern Ireland),Europe,Northern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS,:,376,314,307,350,408,:,463,:
9,HRV,Croatia,Europe,Southern Europe,Persons held unsentenced,by length of detention,Unsentenced for less than 12 months,Counts,CTS,:,:,:,:,833,909,1149,1152,:


In [ ]:
'''
headerOBJ = D2_totalPersHeldCat2.columns
headerD2_2 = []
for item in headerOBJ:
    headerD2_2.append(item)
    
# specify the file path and name to extract the different dfs
filename = 'D2.22.csv'

# define the desired formatting options
format_options = {'sep': ';',  # use semicolon as delimiter
                  'index': False,  # don't include index column
                  'float_format': '%.2f',  # format float values to 2 decimal places
                  'header': headerD2_2,  # rename headers
                  'encoding': 'utf-8'}  # specify encoding type

# export the dataframe to CSV using the formatting options
D2_totalPersHeldCat2.to_csv(filename,**format_options)
'''

<hr>

## D3 cleaning

Contents:
##### Unit
- **P_HTHAB**: 
- **NR**:
##### Freq
- **A**: Annual
##### Categories of data
- **PRIS_ACT_CAP**:
- **PRIS_OFF_CAPP**

In [87]:
df3 = pd.read_table('../raw_data/D3_prison_capacity.tsv', sep='\t', header=0)
df3

,"freq,indic_cr,unit,geo\TIME_PERIOD",2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,"A,PRIS_ACT_CAP,NR,AL",4659,4618,4998,5689,5981,6031,5674,5316,5045,4614
1,"A,PRIS_ACT_CAP,NR,AT",8770,8805,8862,8692,8665,8619,8852,9163,9072,8488
2,"A,PRIS_ACT_CAP,NR,BA",2757,:,2898,2825,2730,2832,2820,4377,4482,4097
3,"A,PRIS_ACT_CAP,NR,BE",11040,11183,11818,:,10994,:,10471,10261,10559,10381
4,"A,PRIS_ACT_CAP,NR,BG",9885,9493,8834,7870,7408,7345,6988,6651,6448,6251
...,...,...,...,...,...,...,...,...,...,...,...
157,"A,PRIS_OFF_CAP,P_HTHAB,SK",207.72,209.17,218.30,218.39,206.30,203.33,201.29,211.26,213.29,213.00
158,"A,PRIS_OFF_CAP,P_HTHAB,TR",158.37,190.65,203.78,212.77,228.63,257.39,261.58,264.65,292.33,294.87
159,"A,PRIS_OFF_CAP,P_HTHAB,UKC-L",164.86,164.86,155.83,156.95,154.52,153.39,151.72,150.08,:,:
160,"A,PRIS_OFF_CAP,P_HTHAB,UKN",97.54,97.03,104.29,104.18,97.07,:,:,98.88,:,:


In [89]:
to_sep = pd.DataFrame(df3.iloc[:,0])
to_sep

,"freq,indic_cr,unit,geo\TIME_PERIOD"
0,"A,PRIS_ACT_CAP,NR,AL"
1,"A,PRIS_ACT_CAP,NR,AT"
2,"A,PRIS_ACT_CAP,NR,BA"
3,"A,PRIS_ACT_CAP,NR,BE"
4,"A,PRIS_ACT_CAP,NR,BG"
...,...
157,"A,PRIS_OFF_CAP,P_HTHAB,SK"
158,"A,PRIS_OFF_CAP,P_HTHAB,TR"
159,"A,PRIS_OFF_CAP,P_HTHAB,UKC-L"
160,"A,PRIS_OFF_CAP,P_HTHAB,UKN"


In [90]:
to_sep[['freq','indic_cr','unit','geo']] = to_sep["freq,indic_cr,unit,geo\TIME_PERIOD"].str.split(",", expand = True)
to_sep

,"freq,indic_cr,unit,geo\TIME_PERIOD",freq,indic_cr,unit,geo
0,"A,PRIS_ACT_CAP,NR,AL",A,PRIS_ACT_CAP,NR,AL
1,"A,PRIS_ACT_CAP,NR,AT",A,PRIS_ACT_CAP,NR,AT
2,"A,PRIS_ACT_CAP,NR,BA",A,PRIS_ACT_CAP,NR,BA
3,"A,PRIS_ACT_CAP,NR,BE",A,PRIS_ACT_CAP,NR,BE
4,"A,PRIS_ACT_CAP,NR,BG",A,PRIS_ACT_CAP,NR,BG
...,...,...,...,...,...
157,"A,PRIS_OFF_CAP,P_HTHAB,SK",A,PRIS_OFF_CAP,P_HTHAB,SK
158,"A,PRIS_OFF_CAP,P_HTHAB,TR",A,PRIS_OFF_CAP,P_HTHAB,TR
159,"A,PRIS_OFF_CAP,P_HTHAB,UKC-L",A,PRIS_OFF_CAP,P_HTHAB,UKC-L
160,"A,PRIS_OFF_CAP,P_HTHAB,UKN",A,PRIS_OFF_CAP,P_HTHAB,UKN


In [103]:
df3 = to_sep.merge(df3, left_index=True, right_index=True)
df3 = df3.drop(columns=["freq,indic_cr,unit,geo\TIME_PERIOD_x", "freq,indic_cr,unit,geo\TIME_PERIOD_y"])
df3

/var/folders/yq/sryk517930l66t3h_fdc8w040000gn/T/ipykernel_42994/266459575.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'geo_y', 'unit_y', 'indic_cr_y', 'freq_y'} in the result is deprecated and will raise a MergeError in a future version.
  df3 = to_sep.merge(df3, left_index=True, right_index=True)


,freq_x,indic_cr_x,unit_x,geo_x,freq_y,indic_cr_y,unit_y,geo_y,freq_x,indic_cr_x,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,A,PRIS_ACT_CAP,NR,AL,A,PRIS_ACT_CAP,NR,AL,A,PRIS_ACT_CAP,...,4659,4618,4998,5689,5981,6031,5674,5316,5045,4614
1,A,PRIS_ACT_CAP,NR,AT,A,PRIS_ACT_CAP,NR,AT,A,PRIS_ACT_CAP,...,8770,8805,8862,8692,8665,8619,8852,9163,9072,8488
2,A,PRIS_ACT_CAP,NR,BA,A,PRIS_ACT_CAP,NR,BA,A,PRIS_ACT_CAP,...,2757,:,2898,2825,2730,2832,2820,4377,4482,4097
3,A,PRIS_ACT_CAP,NR,BE,A,PRIS_ACT_CAP,NR,BE,A,PRIS_ACT_CAP,...,11040,11183,11818,:,10994,:,10471,10261,10559,10381
4,A,PRIS_ACT_CAP,NR,BG,A,PRIS_ACT_CAP,NR,BG,A,PRIS_ACT_CAP,...,9885,9493,8834,7870,7408,7345,6988,6651,6448,6251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,A,PRIS_OFF_CAP,P_HTHAB,SK,A,PRIS_OFF_CAP,P_HTHAB,SK,A,PRIS_OFF_CAP,...,207.72,209.17,218.30,218.39,206.30,203.33,201.29,211.26,213.29,213.00
158,A,PRIS_OFF_CAP,P_HTHAB,TR,A,PRIS_OFF_CAP,P_HTHAB,TR,A,PRIS_OFF_CAP,...,158.37,190.65,203.78,212.77,228.63,257.39,261.58,264.65,292.33,294.87
159,A,PRIS_OFF_CAP,P_HTHAB,UKC-L,A,PRIS_OFF_CAP,P_HTHAB,UKC-L,A,PRIS_OFF_CAP,...,164.86,164.86,155.83,156.95,154.52,153.39,151.72,150.08,:,:
160,A,PRIS_OFF_CAP,P_HTHAB,UKN,A,PRIS_OFF_CAP,P_HTHAB,UKN,A,PRIS_OFF_CAP,...,97.54,97.03,104.29,104.18,97.07,:,:,98.88,:,:


In [97]:
df3.reset_index().T.drop_duplicates().T

,index,"freq,indic_cr,unit,geo\TIME_PERIOD",freq_x,indic_cr_x,unit_x,geo_x,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,"A,PRIS_ACT_CAP,NR,AL",A,PRIS_ACT_CAP,NR,AL,4659,4618,4998,5689,5981,6031,5674,5316,5045,4614
1,1,"A,PRIS_ACT_CAP,NR,AT",A,PRIS_ACT_CAP,NR,AT,8770,8805,8862,8692,8665,8619,8852,9163,9072,8488
2,2,"A,PRIS_ACT_CAP,NR,BA",A,PRIS_ACT_CAP,NR,BA,2757,:,2898,2825,2730,2832,2820,4377,4482,4097
3,3,"A,PRIS_ACT_CAP,NR,BE",A,PRIS_ACT_CAP,NR,BE,11040,11183,11818,:,10994,:,10471,10261,10559,10381
4,4,"A,PRIS_ACT_CAP,NR,BG",A,PRIS_ACT_CAP,NR,BG,9885,9493,8834,7870,7408,7345,6988,6651,6448,6251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,157,"A,PRIS_OFF_CAP,P_HTHAB,SK",A,PRIS_OFF_CAP,P_HTHAB,SK,207.72,209.17,218.30,218.39,206.30,203.33,201.29,211.26,213.29,213.00
158,158,"A,PRIS_OFF_CAP,P_HTHAB,TR",A,PRIS_OFF_CAP,P_HTHAB,TR,158.37,190.65,203.78,212.77,228.63,257.39,261.58,264.65,292.33,294.87
159,159,"A,PRIS_OFF_CAP,P_HTHAB,UKC-L",A,PRIS_OFF_CAP,P_HTHAB,UKC-L,164.86,164.86,155.83,156.95,154.52,153.39,151.72,150.08,:,:
160,160,"A,PRIS_OFF_CAP,P_HTHAB,UKN",A,PRIS_OFF_CAP,P_HTHAB,UKN,97.54,97.03,104.29,104.18,97.07,:,:,98.88,:,:


In [98]:
cleandf = df3.drop(columns=['freq,indic_cr,unit,geo\TIME_PERIOD']) 

cleandf

,freq_x,indic_cr_x,unit_x,geo_x,freq_y,indic_cr_y,unit_y,geo_y,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,A,PRIS_ACT_CAP,NR,AL,A,PRIS_ACT_CAP,NR,AL,4659,4618,4998,5689,5981,6031,5674,5316,5045,4614
1,A,PRIS_ACT_CAP,NR,AT,A,PRIS_ACT_CAP,NR,AT,8770,8805,8862,8692,8665,8619,8852,9163,9072,8488
2,A,PRIS_ACT_CAP,NR,BA,A,PRIS_ACT_CAP,NR,BA,2757,:,2898,2825,2730,2832,2820,4377,4482,4097
3,A,PRIS_ACT_CAP,NR,BE,A,PRIS_ACT_CAP,NR,BE,11040,11183,11818,:,10994,:,10471,10261,10559,10381
4,A,PRIS_ACT_CAP,NR,BG,A,PRIS_ACT_CAP,NR,BG,9885,9493,8834,7870,7408,7345,6988,6651,6448,6251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,A,PRIS_OFF_CAP,P_HTHAB,SK,A,PRIS_OFF_CAP,P_HTHAB,SK,207.72,209.17,218.30,218.39,206.30,203.33,201.29,211.26,213.29,213.00
158,A,PRIS_OFF_CAP,P_HTHAB,TR,A,PRIS_OFF_CAP,P_HTHAB,TR,158.37,190.65,203.78,212.77,228.63,257.39,261.58,264.65,292.33,294.87
159,A,PRIS_OFF_CAP,P_HTHAB,UKC-L,A,PRIS_OFF_CAP,P_HTHAB,UKC-L,164.86,164.86,155.83,156.95,154.52,153.39,151.72,150.08,:,:
160,A,PRIS_OFF_CAP,P_HTHAB,UKN,A,PRIS_OFF_CAP,P_HTHAB,UKN,97.54,97.03,104.29,104.18,97.07,:,:,98.88,:,:


In [99]:
cleandf.rename(
    columns={"unit_x":"Unit", "geo_x": "Geo"}, 
    inplace=True)
cleandf

,freq_x,indic_cr_x,Unit,Geo,freq_y,indic_cr_y,unit_y,geo_y,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,A,PRIS_ACT_CAP,NR,AL,A,PRIS_ACT_CAP,NR,AL,4659,4618,4998,5689,5981,6031,5674,5316,5045,4614
1,A,PRIS_ACT_CAP,NR,AT,A,PRIS_ACT_CAP,NR,AT,8770,8805,8862,8692,8665,8619,8852,9163,9072,8488
2,A,PRIS_ACT_CAP,NR,BA,A,PRIS_ACT_CAP,NR,BA,2757,:,2898,2825,2730,2832,2820,4377,4482,4097
3,A,PRIS_ACT_CAP,NR,BE,A,PRIS_ACT_CAP,NR,BE,11040,11183,11818,:,10994,:,10471,10261,10559,10381
4,A,PRIS_ACT_CAP,NR,BG,A,PRIS_ACT_CAP,NR,BG,9885,9493,8834,7870,7408,7345,6988,6651,6448,6251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,A,PRIS_OFF_CAP,P_HTHAB,SK,A,PRIS_OFF_CAP,P_HTHAB,SK,207.72,209.17,218.30,218.39,206.30,203.33,201.29,211.26,213.29,213.00
158,A,PRIS_OFF_CAP,P_HTHAB,TR,A,PRIS_OFF_CAP,P_HTHAB,TR,158.37,190.65,203.78,212.77,228.63,257.39,261.58,264.65,292.33,294.87
159,A,PRIS_OFF_CAP,P_HTHAB,UKC-L,A,PRIS_OFF_CAP,P_HTHAB,UKC-L,164.86,164.86,155.83,156.95,154.52,153.39,151.72,150.08,:,:
160,A,PRIS_OFF_CAP,P_HTHAB,UKN,A,PRIS_OFF_CAP,P_HTHAB,UKN,97.54,97.03,104.29,104.18,97.07,:,:,98.88,:,:


In [ ]:
#exporting data in a csv
cleandf.to_csv('./clean_datasets/D3_clean.csv', index=False)

<hr>

## D4 cleaning

Contents:
##### Unit
- **Percentage**: 
##### Freq
- **A**: Annual
##### Categories of data
- **FBAD**:
- **FGOOD**
- **UNK**
- **VBAD**
- **VGOOD**

In [105]:
D4 = pd.read_csv("../raw_data/D4-percieved-independece-justiceSystem.tsv",sep='\t',header=0)
D4

,"freq,lev_perc,unit,geo\TIME_PERIOD",2016,2017,2018,2019,2020,2021,2022
0,"A,FBAD,PC,AT",13,14,7,8,7,7,8
1,"A,FBAD,PC,BE",19,21,18,17,19,18,23
2,"A,FBAD,PC,BG",35,37,31,35,31,33,30
3,"A,FBAD,PC,CY",19,14,18,15,20,21,25
4,"A,FBAD,PC,CZ",32,31,28,27,25,27,26
...,...,...,...,...,...,...,...,...
205,"A,VG_FG,PC,RO",51,50,47,40,37,51,48
206,"A,VG_FG,PC,SE",77,72,75,79,81,71,74
207,"A,VG_FG,PC,SI",30,35,33,39,42,47,49
208,"A,VG_FG,PC,SK",21,23,29,28,26,28,25


In [106]:
D4.columns

Index(['freq,lev_perc,unit,geo\TIME_PERIOD', '2016 ', '2017 ', '2018 ',
       '2019 ', '2020 ', '2021 ', '2022 '],
      dtype='object')

In [107]:
# why? –––––> To understand the indexing structure 
for items in D4.columns:
    print(items)

first_col = D4.iloc[:,0]
freq = []
lev_per = []
unit = []
geo = []

for row in first_col:
    output_list = []
    word = str()
    #print(row)
    #print(len(row))
    for i in range(len(row)):
        if row[i] != ",":
            word = word + row[i]
            #print(word)
        else:
            output_list.append(word)
            word = str()
    if word != "":
        output_list.append(word)
    freq.append(output_list[0])
    lev_per.append(output_list[1])
    unit.append(output_list[2])
    geo.append(output_list[3])

# total rows = 210
colDict = {'freq':freq,'lev_per':lev_per,'unit':unit,'geo':geo}
D4_firstCol = pd.DataFrame(data=colDict)


#merging 
D4_data = D4.drop(columns=['freq,lev_perc,unit,geo\TIME_PERIOD'])

D4_structured = D4_firstCol.merge(D4_data, left_index=True, right_index=True)
D4_structured.head(10)

freq,lev_perc,unit,geo\TIME_PERIOD
2016 
2017 
2018 
2019 
2020 
2021 
2022 


,freq,lev_per,unit,geo,2016,2017,2018,2019,2020,2021,2022
0,A,FBAD,PC,AT,13,14,7,8,7,7,8
1,A,FBAD,PC,BE,19,21,18,17,19,18,23
2,A,FBAD,PC,BG,35,37,31,35,31,33,30
3,A,FBAD,PC,CY,19,14,18,15,20,21,25
4,A,FBAD,PC,CZ,32,31,28,27,25,27,26
5,A,FBAD,PC,DE,13,12,14,15,11,11,8
6,A,FBAD,PC,DK,6,5,7,5,6,8,6
7,A,FBAD,PC,EE,12,12,11,12,12,8,14
8,A,FBAD,PC,EL,26,25,22,24,24,23,28
9,A,FBAD,PC,ES,38,39,32,37,34,32,35


In [108]:
# check for missing and NAN values
missing_values = D4_structured.isnull().sum()
print(missing_values)

# finding rows with missing data and the associated country
for index,row in D4_structured.iterrows():
    for col,value in row.items():
        if ':' in value:
            print(row['geo'])

freq       0
lev_per    0
unit       0
geo        0
2016       0
2017       0
2018       0
2019       0
2020       0
2021       0
2022       0
dtype: int64
EU28
EU28
EU28
UK
UK
EU28
EU28
EU28
UK
UK
EU28
EU28
EU28
UK
UK
EU28
EU28
EU28
UK
UK
EU28
EU28
EU28
UK
UK
EU28
EU28
EU28
UK
UK
EU28
EU28
EU28
UK
UK


After checking for null values we see that the D4 dataset has no null values in any of its columns. However, we do have `:` as a placeholder for missing values and we observer that it's missing in namely two locations:
- EU28
- UK

> We will **decide later** what to do with the rows with the placeholder values

In [109]:
# finding rows with breaks in the time series and the associated country
for index,row in D4_structured.iterrows():
    for col,value in row.items():
        if 'b' in value:
            print(row['geo'])

EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020


In [110]:
# finding rows with estimated values and the associated country
for index,row in D4_structured.iterrows():
    for col,value in row.items():
        if 'e' in value:
            print(row['geo'])

EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020
EU27_2020


In [111]:
# finding all unique countries in the DF
loc = []
for index,row in D4_structured.iterrows():
    for col,value in row.items():
        if col == 'geo' and row['geo'] not in loc:
            loc.append(row['geo'])

print(loc)
print(len(loc))

['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'EU27_2020', 'EU28', 'FI', 'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']
30


### Important!
After we create the grouped object for each location, we can use the stored values of the location codes to create new Dataframes with only the data of those locations. Below I have created the dataframe for Belgium using the code `BE`.

In [112]:
loc2 = []
# deleting estimated values for EU27 and EU28
for index,row in D4_structured.iterrows():
    for col,value in row.items():
        if col ==  'geo' and 'EU27' in row['geo']:
            loc2.append(index)

#deleting the 'freq' and 'unit' as it is the same for every data entry
D4_structured = D4_structured.drop(columns=['freq','unit'])
D4_structured.head(10)

,lev_per,geo,2016,2017,2018,2019,2020,2021,2022
0,FBAD,AT,13,14,7,8,7,7,8
1,FBAD,BE,19,21,18,17,19,18,23
2,FBAD,BG,35,37,31,35,31,33,30
3,FBAD,CY,19,14,18,15,20,21,25
4,FBAD,CZ,32,31,28,27,25,27,26
5,FBAD,DE,13,12,14,15,11,11,8
6,FBAD,DK,6,5,7,5,6,8,6
7,FBAD,EE,12,12,11,12,12,8,14
8,FBAD,EL,26,25,22,24,24,23,28
9,FBAD,ES,38,39,32,37,34,32,35


In [113]:
# group the dataframe by column 'geo'
D4_structured2 = D4_structured.groupby(['geo'])

#fetch the data of any location using the country code store in the loc varaible
D4_structured2.get_group('UK')

,lev_per,geo,2016,2017,2018,2019,2020,2021,2022
29,FBAD,UK,12,11,13,10,12,:,:
59,FGOOD,UK,52,45,48,49,54,:,:
89,UNK,UK,12,13,12,11,8,:,:
119,VBAD,UK,6,5,7,9,9,:,:
149,VB_FB,UK,18,16,20,19,21,:,:
179,VGOOD,UK,18,26,20,21,17,:,:
209,VG_FG,UK,70,71,68,68,71,:,:


In [115]:
#writing the loop to iterate over all the country group objects and mergeing them into one df
D4_final = pd.DataFrame()
for item in D4_structured2.groups:
    DF = pd.DataFrame()
    DF = D4_structured2.get_group(item)
    D4_final = pd.concat([D4_final,DF])

D4_final    

,lev_per,geo,2016,2017,2018,2019,2020,2021,2022
0,FBAD,AT,13,14,7,8,7,7,8
30,FGOOD,AT,59,58,58,65,62,62,59
60,UNK,AT,7,6,10,7,5,7,7
90,VBAD,AT,3,2,2,2,2,2,3
120,VB_FB,AT,16,16,9,10,9,9,10
...,...,...,...,...,...,...,...,...,...
89,UNK,UK,12,13,12,11,8,:,:
119,VBAD,UK,6,5,7,9,9,:,:
149,VB_FB,UK,18,16,20,19,21,:,:
179,VGOOD,UK,18,26,20,21,17,:,:


In [116]:
headerOBJ = D4_final.columns
header = []
for item in headerOBJ:
    header.append(item)
    
print(header)

['lev_per', 'geo', '2016 ', '2017 ', '2018 ', '2019 ', '2020 ', '2021 ', '2022 ']


In [ ]:
'''
# specify the file path and name
filename = 'D4.csv'

# define the desired formatting options
format_options = {'sep': ';',  # use semicolon as delimiter
                  'index': False,  # don't include index column
                  'float_format': '%.2f',  # format float values to 2 decimal places
                  'header': header,  # rename headers
                  'encoding': 'utf-8'}  # specify encoding type

# export the dataframe to CSV using the formatting options
D4_final.to_csv(filename,**format_options) 
'''

<hr>

## D5 cleaning

Contents:
##### Unit
- **NR**: rank in corruption perception index
##### Freq
- **A**: Annual

In [117]:
df5 = pd.read_table('../raw_data/D5_CPI_from_eurostat.tsv', sep='\t', header=0)
df5

,"freq,unit,geo\TIME_PERIOD",2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,"A,NR,AL",33,31,33,36,39,38,36,35,36,35,36
1,"A,NR,AT",69,69,72,76,75,75,76,77,76,74,71
2,"A,NR,BE",75,75,76,77,77,75,75,75,76,73,73
3,"A,NR,BG",41,41,43,41,41,43,42,43,44,42,43
4,"A,NR,CH",86,85,86,86,86,85,85,85,85,84,82
5,"A,NR,CY",66,63,63,61,55,57,59,58,57,53,52
6,"A,NR,CZ",49,48,51,56,55,57,59,56,54,54,56
7,"A,NR,DE",79,78,79,81,81,81,80,80,80,80,79
8,"A,NR,DK",90,91,92,91,90,88,88,87,88,88,90
9,"A,NR,EE",64,68,69,70,70,71,73,74,75,74,74


In [118]:
to_sep = pd.DataFrame(df5.iloc[:,0])
to_sep

,"freq,unit,geo\TIME_PERIOD"
0,"A,NR,AL"
1,"A,NR,AT"
2,"A,NR,BE"
3,"A,NR,BG"
4,"A,NR,CH"
5,"A,NR,CY"
6,"A,NR,CZ"
7,"A,NR,DE"
8,"A,NR,DK"
9,"A,NR,EE"


In [119]:
to_sep[['freq', 'unit','geo']] = to_sep["freq,unit,geo\TIME_PERIOD"].str.split(",", expand = True)
to_sep

,"freq,unit,geo\TIME_PERIOD",freq,unit,geo
0,"A,NR,AL",A,NR,AL
1,"A,NR,AT",A,NR,AT
2,"A,NR,BE",A,NR,BE
3,"A,NR,BG",A,NR,BG
4,"A,NR,CH",A,NR,CH
5,"A,NR,CY",A,NR,CY
6,"A,NR,CZ",A,NR,CZ
7,"A,NR,DE",A,NR,DE
8,"A,NR,DK",A,NR,DK
9,"A,NR,EE",A,NR,EE


In [120]:
df5 = to_sep.merge(df5, left_index=True, right_index=True)
df5 = df5.drop(columns=["freq,unit,geo\TIME_PERIOD_x", "freq,unit,geo\TIME_PERIOD_y"])
df5

,freq,unit,geo,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,A,NR,AL,33,31,33,36,39,38,36,35,36,35,36
1,A,NR,AT,69,69,72,76,75,75,76,77,76,74,71
2,A,NR,BE,75,75,76,77,77,75,75,75,76,73,73
3,A,NR,BG,41,41,43,41,41,43,42,43,44,42,43
4,A,NR,CH,86,85,86,86,86,85,85,85,85,84,82
5,A,NR,CY,66,63,63,61,55,57,59,58,57,53,52
6,A,NR,CZ,49,48,51,56,55,57,59,56,54,54,56
7,A,NR,DE,79,78,79,81,81,81,80,80,80,80,79
8,A,NR,DK,90,91,92,91,90,88,88,87,88,88,90
9,A,NR,EE,64,68,69,70,70,71,73,74,75,74,74


In [121]:
cleandf = df5.drop(columns=['freq'])
cleandf

,unit,geo,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,NR,AL,33,31,33,36,39,38,36,35,36,35,36
1,NR,AT,69,69,72,76,75,75,76,77,76,74,71
2,NR,BE,75,75,76,77,77,75,75,75,76,73,73
3,NR,BG,41,41,43,41,41,43,42,43,44,42,43
4,NR,CH,86,85,86,86,86,85,85,85,85,84,82
5,NR,CY,66,63,63,61,55,57,59,58,57,53,52
6,NR,CZ,49,48,51,56,55,57,59,56,54,54,56
7,NR,DE,79,78,79,81,81,81,80,80,80,80,79
8,NR,DK,90,91,92,91,90,88,88,87,88,88,90
9,NR,EE,64,68,69,70,70,71,73,74,75,74,74


In [ ]:
'''
#exporting the data in a csv
cleandf.to_csv('./clean_datasets/D5_clean.csv', index=False)
'''

<hr>

## D6 cleaning

Contents:
##### Unit
- **PC_GDP**: Percentage of Gross Domestic Product
- **MIO_EUR**: Millior Euro
##### Freq
- **A**: Annual
##### Sector
- **S13**: General Government
- **S1311**: Central Government
- **S1312**: State Government
- **S1313**: Local Government
- **S1314**: Social Security Funds
#### na_item (National Account Indicator)
- **TE**: Total general government ecpenditure
##### cofog_99 (Classification of the functions of government)
- View section "Filter gov functions"

In [122]:
df6 = pd.read_table('../raw_data/D6_general_governments_expenditure.tsv', sep='\t', header=0)
df6

,"freq,unit,sector,cofog99,na_item,geo\TIME_PERIOD",2015,2016,2017,2018,2019,2020,2021
0,"A,MIO_EUR,S13,GF01,TE,AT",23388.9,23109.4,22172.9,22773.2,22851.4,23419.4,23502.8
1,"A,MIO_EUR,S13,GF01,TE,BE",33422.2,34096.0,31904.8,32561.0,32823.9,33771.3,34976.9
2,"A,MIO_EUR,S13,GF01,TE,BG",1725.8,1729.7,1665.6,2058.0,2198.9,2283.2,2489.6
3,"A,MIO_EUR,S13,GF01,TE,CH",28712.9,27810.3,28326.6,27125.1,27922.2,29662.3,29326.7
4,"A,MIO_EUR,S13,GF01,TE,CY",1623.8,1486.6,1487.7,1508.8,1662.6,1639.3,1647.6
...,...,...,...,...,...,...,...,...
23635,"A,PC_GDP,S1314,TOTAL,TE,PT",12.8,12.5,12.1,12.0,12.1,14.5,13.9 p
23636,"A,PC_GDP,S1314,TOTAL,TE,RO",11.1,10.5,10.4,10.5,10.8,12.8,12.2
23637,"A,PC_GDP,S1314,TOTAL,TE,SE",6.3,6.5,6.5,6.4,6.3,6.6,6.2
23638,"A,PC_GDP,S1314,TOTAL,TE,SI",19.1,18.8,18.1,17.9,17.8,19.5,18.7


In [123]:
to_sep = pd.DataFrame(df6.iloc[:,0])
to_sep

,"freq,unit,sector,cofog99,na_item,geo\TIME_PERIOD"
0,"A,MIO_EUR,S13,GF01,TE,AT"
1,"A,MIO_EUR,S13,GF01,TE,BE"
2,"A,MIO_EUR,S13,GF01,TE,BG"
3,"A,MIO_EUR,S13,GF01,TE,CH"
4,"A,MIO_EUR,S13,GF01,TE,CY"
...,...
23635,"A,PC_GDP,S1314,TOTAL,TE,PT"
23636,"A,PC_GDP,S1314,TOTAL,TE,RO"
23637,"A,PC_GDP,S1314,TOTAL,TE,SE"
23638,"A,PC_GDP,S1314,TOTAL,TE,SI"


In [124]:
to_sep[['freq','unit','sector','cofog99','na_item','geo']] = to_sep["freq,unit,sector,cofog99,na_item,geo\TIME_PERIOD"].str.split(",", expand=True)
to_sep

,"freq,unit,sector,cofog99,na_item,geo\TIME_PERIOD",freq,unit,sector,cofog99,na_item,geo
0,"A,MIO_EUR,S13,GF01,TE,AT",A,MIO_EUR,S13,GF01,TE,AT
1,"A,MIO_EUR,S13,GF01,TE,BE",A,MIO_EUR,S13,GF01,TE,BE
2,"A,MIO_EUR,S13,GF01,TE,BG",A,MIO_EUR,S13,GF01,TE,BG
3,"A,MIO_EUR,S13,GF01,TE,CH",A,MIO_EUR,S13,GF01,TE,CH
4,"A,MIO_EUR,S13,GF01,TE,CY",A,MIO_EUR,S13,GF01,TE,CY
...,...,...,...,...,...,...,...
23635,"A,PC_GDP,S1314,TOTAL,TE,PT",A,PC_GDP,S1314,TOTAL,TE,PT
23636,"A,PC_GDP,S1314,TOTAL,TE,RO",A,PC_GDP,S1314,TOTAL,TE,RO
23637,"A,PC_GDP,S1314,TOTAL,TE,SE",A,PC_GDP,S1314,TOTAL,TE,SE
23638,"A,PC_GDP,S1314,TOTAL,TE,SI",A,PC_GDP,S1314,TOTAL,TE,SI


In [125]:
df6 = to_sep.merge(df6, left_index=True, right_index=True)
df6 = df6.drop(columns=['freq,unit,sector,cofog99,na_item,geo\TIME_PERIOD_x','freq,unit,sector,cofog99,na_item,geo\TIME_PERIOD_y'])
df6

,freq,unit,sector,cofog99,na_item,geo,2015,2016,2017,2018,2019,2020,2021
0,A,MIO_EUR,S13,GF01,TE,AT,23388.9,23109.4,22172.9,22773.2,22851.4,23419.4,23502.8
1,A,MIO_EUR,S13,GF01,TE,BE,33422.2,34096.0,31904.8,32561.0,32823.9,33771.3,34976.9
2,A,MIO_EUR,S13,GF01,TE,BG,1725.8,1729.7,1665.6,2058.0,2198.9,2283.2,2489.6
3,A,MIO_EUR,S13,GF01,TE,CH,28712.9,27810.3,28326.6,27125.1,27922.2,29662.3,29326.7
4,A,MIO_EUR,S13,GF01,TE,CY,1623.8,1486.6,1487.7,1508.8,1662.6,1639.3,1647.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23635,A,PC_GDP,S1314,TOTAL,TE,PT,12.8,12.5,12.1,12.0,12.1,14.5,13.9 p
23636,A,PC_GDP,S1314,TOTAL,TE,RO,11.1,10.5,10.4,10.5,10.8,12.8,12.2
23637,A,PC_GDP,S1314,TOTAL,TE,SE,6.3,6.5,6.5,6.4,6.3,6.6,6.2
23638,A,PC_GDP,S1314,TOTAL,TE,SI,19.1,18.8,18.1,17.9,17.8,19.5,18.7


## Filter gov functions

The available values are:
- [<font color=crimson>**TOTAL**</font>] Total
- [<font color=crimson>**GF01**</font>] General public services
- [<font color=crimson>**GF0101**</font>] Executive and legislative organs, financial and fiscal affairs, external affairs
- [<font color=crimson>**GF0102**</font>] Foreign economic aid
- [<font color=crimson>**GF0103**</font>] General services
- [<font color=crimson>**GF0104**</font>] Basic research
- [<font color=crimson>**GF0105**</font>] R&D General public services
- [<font color=crimson>**GF0106**</font>] General public services n.e.c.
- [<font color=crimson>**GF0107**</font>] Public debt transactions
- [<font color=crimson>**GF0108**</font>] Transfers of a general character between different levels of government
- [<font color=crimson>**GF02**</font>] Defence
- [<font color=crimson>**GF0201**</font>] Military defence
- [<font color=crimson>**GF0202**</font>] Civil defence
- [<font color=crimson>**GF0203**</font>] Foreign military aid
- [<font color=crimson>**GF0204**</font>] R&D Defence
- [<font color=crimson>**GF0205**</font>] Defence n.e.c.
- [<font color=crimson>**GF03**</font>] Public order and safety
- [<font color=crimson>**GF0301**</font>] Police services
- [<font color=crimson>**GF0302**</font>] Fire-protection services
- [<font color=crimson>**GF0303**</font>] Law courts
- [<font color=crimson>**GF0304**</font>] Prisons
- [<font color=crimson>**GF0305**</font>] R&D Public order and safety
- [<font color=crimson>**GF0306**</font>] Public order and safety n.e.c.
- [<font color=crimson>**GF04**</font>] Economic affairs
- [<font color=crimson>**GF0401**</font>] General economic, commercial and labour affairs
- [<font color=crimson>**GF0402**</font>] Agriculture, forestry, fishing and hunting
- [<font color=crimson>**GF0403**</font>] Fuel and energy
- [<font color=crimson>**GF0404**</font>] Mining, manufacturing and construction
- [<font color=crimson>**GF0405**</font>] Transport
- [<font color=crimson>**GF0406**</font>] Communication
- [<font color=crimson>**GF0407**</font>] Other industries
- [<font color=crimson>**GF0408**</font>] R&D Economic affairs
- [<font color=crimson>**GF0409**</font>] Economic affairs n.e.c. 
- [<font color=crimson>**GF05**</font>] Environmental protection
- [<font color=crimson>**GF0501**</font>] Waste management
- [<font color=crimson>**GF0502**</font>] Waste water management
- [<font color=crimson>**GF0503**</font>] Pollution abatement
- [<font color=crimson>**GF0504**</font>] Protection of biodiversity and landscape
- [<font color=crimson>**GF0505**</font>] R&D Environmental protection
- [<font color=crimson>**GF0506**</font>] Environmental protection n.e.c.
- [<font color=crimson>**GF06**</font>] Housing and community amenities 
- [<font color=crimson>**GF0601**</font>] Housing development 
- [<font color=crimson>**GF0602**</font>] Community development 
- [<font color=crimson>**GF0603**</font>] Water supply
- [<font color=crimson>**GF0604**</font>] Street lighting
- [<font color=crimson>**GF0605**</font>] R&D Housing and community amenities
- [<font color=crimson>**GF0606**</font>] Housing and community amenities n.e.c.
- [<font color=crimson>**GF07**</font>] Health
- [<font color=crimson>**GF0701**</font>] Medical products, appliances and equipment
- [<font color=crimson>**GF0702**</font>] Outpatient services
- [<font color=crimson>**GF0703**</font>] Hospital services
- [<font color=crimson>**GF0704**</font>] Public health services
- [<font color=crimson>**GF0705**</font>] R&D Health
- [<font color=crimson>**GF0706**</font>] Health n.e.c.
- [<font color=crimson>**GF08**</font>] Recreation, culture and religion
- [<font color=crimson>**GF0801**</font>] Recreational and sporting services
- [<font color=crimson>**GF0802**</font>] Cultural services 
- [<font color=crimson>**GF0803**</font>] Broadcasting and publishing services
- [<font color=crimson>**GF0804**</font>] Religious and other community services
- [<font color=crimson>**GF0805**</font>] R&D Recreation, culture and religion
- [<font color=crimson>**GF0806**</font>] Recreation, culture and religion n.e.c.
- [<font color=crimson>**GF09**</font>] Education
- [<font color=crimson>**GF0901**</font>] Pre-primary and primary education
- [<font color=crimson>**GF0902**</font>] Secondary education
- [<font color=crimson>**GF0903**</font>] Post-secondary non-tertiary education
- [<font color=crimson>**GF0904**</font>] Tertiary education 
- [<font color=crimson>**GF0905**</font>] Education not definable by level
- [<font color=crimson>**GF0906**</font>] Subsidiary services to education
- [<font color=crimson>**GF0907**</font>] R&D Education
- [<font color=crimson>**GF0908**</font>] Education n.e.c.
- [<font color=crimson>**GF10**</font>] Social protection
- [<font color=crimson>**GF1001**</font>] Sickness and disability
- [<font color=crimson>**GF1002**</font>] Old age
- [<font color=crimson>**GF1003**</font>] Survivors
- [<font color=crimson>**GF1004**</font>] Family and children
- [<font color=crimson>**GF1005**</font>] Unemployment
- [<font color=crimson>**GF1006**</font>] Housing
- [<font color=crimson>**GF1007**</font>] Social exclusion n.e.c.
- [<font color=crimson>**GF1008**</font>] R&D Social protection
- [<font color=crimson>**GF1009**</font>] Social protection n.e.c.

We filter the dataframe's rows to maintain only the ones related to our analysis, meaning:
- [<font color=crimson>**GF03**</font>] Public order and safety
- [<font color=crimson>**GF0301**</font>] Police services
- [<font color=crimson>**GF0303**</font>] Law courts
- [<font color=crimson>**GF0304**</font>] Prisons
- [<font color=crimson>**GF0305**</font>] R&D Public order and safety
- [<font color=crimson>**GF0306**</font>] Public order and safety n.e.c.

In [126]:
filtered_df = df6.query("cofog99 == 'GF03' or cofog99 == 'GF0301' or cofog99 == 'GF0303' or cofog99 == 'GF0304' or cofog99 == 'GF0305' or cofog99 == 'GF0306'")
filtered_df

,freq,unit,sector,cofog99,na_item,geo,2015,2016,2017,2018,2019,2020,2021
495,A,MIO_EUR,S13,GF03,TE,AT,4624.5,4805.4,5039.2,5226.3,5301.8,5436.8,5695.6
496,A,MIO_EUR,S13,GF03,TE,BE,7166.7,7298.0,7512.9,7965.6,8075.3,8298.4,8811.8
497,A,MIO_EUR,S13,GF03,TE,BG,1279.4,1113.4,1296.9,1378.0,1633.0,1670.3,1914.1
498,A,MIO_EUR,S13,GF03,TE,CH,10327.2,10173.8,10196.3,10105.3,10731.1,11289.0,11320.2
499,A,MIO_EUR,S13,GF03,TE,CY,304.9,309.0,329.3,361.8,379.8,400.6,410.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21971,A,PC_GDP,S1314,GF0306,TE,PT,: z,: z,: z,: z,: z,: z,: z
21972,A,PC_GDP,S1314,GF0306,TE,RO,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21973,A,PC_GDP,S1314,GF0306,TE,SE,: z,: z,: z,: z,: z,: z,: z
21974,A,PC_GDP,S1314,GF0306,TE,SI,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Since the frequency is annual for all the entries, we can delete the resective column. The same is done for `na_item`.

In [127]:
filtered_df = filtered_df.drop(columns=['freq', 'na_item'])
filtered_df

,unit,sector,cofog99,geo,2015,2016,2017,2018,2019,2020,2021
495,MIO_EUR,S13,GF03,AT,4624.5,4805.4,5039.2,5226.3,5301.8,5436.8,5695.6
496,MIO_EUR,S13,GF03,BE,7166.7,7298.0,7512.9,7965.6,8075.3,8298.4,8811.8
497,MIO_EUR,S13,GF03,BG,1279.4,1113.4,1296.9,1378.0,1633.0,1670.3,1914.1
498,MIO_EUR,S13,GF03,CH,10327.2,10173.8,10196.3,10105.3,10731.1,11289.0,11320.2
499,MIO_EUR,S13,GF03,CY,304.9,309.0,329.3,361.8,379.8,400.6,410.0
...,...,...,...,...,...,...,...,...,...,...,...
21971,PC_GDP,S1314,GF0306,PT,: z,: z,: z,: z,: z,: z,: z
21972,PC_GDP,S1314,GF0306,RO,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21973,PC_GDP,S1314,GF0306,SE,: z,: z,: z,: z,: z,: z,: z
21974,PC_GDP,S1314,GF0306,SI,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We also include only the General Government Sector `S13` as a reference for the general expenditure.

In [128]:
complete_df = filtered_df.query("sector == 'S13'").reset_index(drop=True)
complete_df

,unit,sector,cofog99,geo,2015,2016,2017,2018,2019,2020,2021
0,MIO_EUR,S13,GF03,AT,4624.5,4805.4,5039.2,5226.3,5301.8,5436.8,5695.6
1,MIO_EUR,S13,GF03,BE,7166.7,7298.0,7512.9,7965.6,8075.3,8298.4,8811.8
2,MIO_EUR,S13,GF03,BG,1279.4,1113.4,1296.9,1378.0,1633.0,1670.3,1914.1
3,MIO_EUR,S13,GF03,CH,10327.2,10173.8,10196.3,10105.3,10731.1,11289.0,11320.2
4,MIO_EUR,S13,GF03,CY,304.9,309.0,329.3,361.8,379.8,400.6,410.0
...,...,...,...,...,...,...,...,...,...,...,...
391,PC_GDP,S13,GF0306,PT,0.2,0.1,0.1,0.1,0.1,0.1,0.1 p
392,PC_GDP,S13,GF0306,RO,0.3,0.3,0.2,0.2,0.3,0.3,0.3
393,PC_GDP,S13,GF0306,SE,0.0,0.0,0.0,0.0,0.0,0.0,0.0
394,PC_GDP,S13,GF0306,SI,0.1,0.1,0.1,0.1,0.1,0.1,0.1


We can also split the dataset in two according to the unit used to measure the expenditure so that if we need them separatedly we already have them.

In [129]:
df_by_million_euros = complete_df.query("unit == 'MIO_EUR'").reset_index(drop=True)
df_by_million_euros

,unit,sector,cofog99,geo,2015,2016,2017,2018,2019,2020,2021
0,MIO_EUR,S13,GF03,AT,4624.5,4805.4,5039.2,5226.3,5301.8,5436.8,5695.6
1,MIO_EUR,S13,GF03,BE,7166.7,7298.0,7512.9,7965.6,8075.3,8298.4,8811.8
2,MIO_EUR,S13,GF03,BG,1279.4,1113.4,1296.9,1378.0,1633.0,1670.3,1914.1
3,MIO_EUR,S13,GF03,CH,10327.2,10173.8,10196.3,10105.3,10731.1,11289.0,11320.2
4,MIO_EUR,S13,GF03,CY,304.9,309.0,329.3,361.8,379.8,400.6,410.0
...,...,...,...,...,...,...,...,...,...,...,...
193,MIO_EUR,S13,GF0306,PT,270.4,210.0,224.1,229.9,242.3,219.5,232.3 p
194,MIO_EUR,S13,GF0306,RO,556.9,444.5,439.6,485.0,574.9,612.5,662.4
195,MIO_EUR,S13,GF0306,SE,43.8,51.0,63.7,79.4,57.3,52.8,54.4
196,MIO_EUR,S13,GF0306,SI,38.9,40.6,48.3,48.4,52.0,53.3,62.2


In [130]:
df_by_gdp_percentage = complete_df.query("unit == 'PC_GDP'").reset_index(drop=True)
df_by_gdp_percentage

,unit,sector,cofog99,geo,2015,2016,2017,2018,2019,2020,2021
0,PC_GDP,S13,GF03,AT,1.3,1.3,1.4,1.4,1.3,1.4,1.4
1,PC_GDP,S13,GF03,BE,1.7,1.7,1.7,1.7,1.7,1.8,1.8
2,PC_GDP,S13,GF03,BG,2.8,2.3,2.5,2.5,2.7,2.7,2.7
3,PC_GDP,S13,GF03,CH,1.7,1.6,1.7,1.6,1.7,1.7,1.7
4,PC_GDP,S13,GF03,CY,1.7,1.6,1.6,1.7,1.6,1.8,1.7
...,...,...,...,...,...,...,...,...,...,...,...
193,PC_GDP,S13,GF0306,PT,0.2,0.1,0.1,0.1,0.1,0.1,0.1 p
194,PC_GDP,S13,GF0306,RO,0.3,0.3,0.2,0.2,0.3,0.3,0.3
195,PC_GDP,S13,GF0306,SE,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,PC_GDP,S13,GF0306,SI,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [ ]:
'''
#exporting the data in csv
df_by_gdp_percentage.to_csv('../processed_data/clean_datasets/D6_clean.csv', index=False)
'''